# Are some train routes more likely to experience delays than others? Visualize and proof statistically.

In [23]:
import src.data_processing.data as data
import numpy as np
import pandas as pd

In [24]:
trips = data.full_trips
# trips = trips.dropna(subset=['Time of real arrival', 'Time of planned arrival']) # we cant calculate the delay: delete them
trips[["Delay time"]].describe()

,Delay time
count,440943.000000
mean,122.892338
std,350.910273
min,0.000000
25%,0.000000
50%,31.000000
75%,123.000000
max,24536.000000


In [25]:
trips.sort_values(by = "Delay time" , ascending = False)

,Unnamed: 0,Date of departure,Train number,Relation,Railway operators,Railway of departure,Time of real arrival,Time of real departure,Time of planned arrival,Time of planned departure,Direction of relation,Name of the stop,Railway of arrival,Date of planned arrival,Date of planned departure,Date of real arrival,Date of real departure,Delay time
9689,9689,2020-09-22,1785,L B1-1,SNCB/NMBS,NaN,2020-09-22 21:19:56,NaT,2020-09-22 14:31:00,NaT,L B1-1: ANTWERPEN-CENTRAAL -> NIVELLES,vilvoorde,27,2020-09-22,NaT,2020-09-22,NaT,24536.0
61013,61013,2020-09-22,3485,L B7,SNCB/NMBS,NaN,2020-09-22 21:20:16,NaT,2020-09-22 14:36:00,NaT,L B7: MECHELEN -> HALLE,vilvoorde,27,2020-09-22,NaT,2020-09-22,NaT,24256.0
42349,42349,2020-09-21,324,INT,SNCB/NMBS,37,2020-09-21 14:42:47,2020-09-21 14:42:47,2020-09-21 08:50:00,2020-09-21 08:50:00,INT: WIEN HBF -> BRUSSEL-ZUID,hergenrath,37,2020-09-21,2020-09-21,2020-09-21,2020-09-21,21167.0
1335,1335,2020-09-21,324,INT,SNCB/NMBS,37,2020-09-21 14:46:53,2020-09-21 14:49:16,2020-09-21 08:56:00,2020-09-21 09:11:00,INT: WIEN HBF -> BRUSSEL-ZUID,welkenraedt,37,2020-09-21,2020-09-21,2020-09-21,2020-09-21,21053.0
42352,42352,2020-09-21,324,INT,SNCB/NMBS,37,2020-09-21 14:54:16,2020-09-21 14:54:16,2020-09-21 09:16:00,2020-09-21 09:16:00,INT: WIEN HBF -> BRUSSEL-ZUID,dolhain-gileppe,37,2020-09-21,2020-09-21,2020-09-21,2020-09-21,20296.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45426,45426,2020-09-25,2440,IC 18,SNCB/NMBS,125,2020-09-25 19:07:37,2020-09-25 19:07:37,2020-09-25 19:08:00,2020-09-25 19:08:00,IC 18: LIEGE-SAINT-LAMBERT -> BRUSSEL-ZUID,amay,125,2020-09-25,2020-09-25,2020-09-25,2020-09-25,0.0
39932,39932,2020-09-22,1717,IC 14,SNCB/NMBS,96,2020-09-22 18:00:55,2020-09-22 18:00:55,2020-09-22 18:01:00,2020-09-22 18:01:00,IC 14: QUIEVRAIN -> LIEGE-GUILLEMINS,masnuy-saint-pierre,96,2020-09-22,2020-09-22,2020-09-22,2020-09-22,0.0
25814,25814,2020-09-26,537,IC 01,SNCB/NMBS,50A,2020-09-26 16:49:34,2020-09-26 16:49:34,2020-09-26 16:51:00,2020-09-26 16:51:00,IC 01: EUPEN -> OOSTENDE,beernem,50A,2020-09-26,2020-09-26,2020-09-26,2020-09-26,0.0
45429,45429,2020-09-25,2440,IC 18,SNCB/NMBS,125,2020-09-25 19:11:37,2020-09-25 19:12:36,2020-09-25 19:12:00,2020-09-25 19:13:00,IC 18: LIEGE-SAINT-LAMBERT -> BRUSSEL-ZUID,huy,125,2020-09-25,2020-09-25,2020-09-25,2020-09-25,0.0


We delete all delay times > 85.000 seconds, which is equal to almost 24 hours.

In [26]:

trips = data.full_trips
for x in trips[['Delay time']]:
    trips.loc[trips[x] > 85000,x] = np.nan
trips[["Delay time"]].describe()

,Delay time
count,440943.000000
mean,122.892338
std,350.910273
min,0.000000
25%,0.000000
50%,31.000000
75%,123.000000
max,24536.000000


We assume that a train route is an array of subsequent train stops

KeyError: 'Day of real arrival'

In [28]:
train_numbers = trips["Train number"].unique()
routes = {}
for train_number in train_numbers:
    routes[train_number] = trips[trips["Train number"] == train_number].sort_values(by= [ "Time of real departure"])

In [32]:
routes_by_day = {}
for train_number in train_numbers:
    routes_by_day[train_number] = [group for _, group in routes[train_number].groupby("Date of departure")]


In [36]:
len(routes_by_day.get(10))

7

Each routes value actually contains multiple times the same set of subsequent stops (1 route per day, 7 days)

In [ ]:
routes.get(18).groupby("Name of the stop").size().reset_index(name="count")
routes.get(18)


Create a dictionary so we can get the "number of delays/ total train stops" per route

In [44]:
# #for train_number in train_numbers:
#     routes[train_number]["Number of delays"] = len(routes.get(train_number)[routes.get(train_number)["Delay time"] > 0])
#     routes[train_number]["Total train stops"] = len(routes.get(train_number))
#     routes[train_number]["Number of delays/ Total train stops"] = routes[train_number]["Number of delays"]/routes[train_number]["Total train stops"]
# routes.get(10)
delay_occurences_per_route = {}
for train_number in train_numbers:
    delay_occurences_per_route [train_number]  = len(routes.get(train_number)[routes.get(train_number)["Delay time"] > 0])/len(routes.get(train_number))

delay_occurences_per_route.get(12)

0.775

In [37]:
percentage_of_delays = {}
for train_number in train_numbers:
    percentage_of_delays[train_number] = []
    for day in routes_by_day.get(train_number):
        percentage_of_delays.get(train_number).append(day[day["Delay time"] > 0].shape[0] / day.shape[0])

In [38]:
percentage_of_delays.get(10)

[1.0, 0.1, 0.25, 1.0, 1.0, 0.85, 0.0]

In [39]:
average_percentage_of_delays = {}
for train_number in train_numbers:
    average_percentage_of_delays[train_number] = sum(percentage_of_delays.get(train_number)) / len(percentage_of_delays.get(train_number))

In [45]:
average_percentage_of_delays.get(12)

0.775

In [ ]:
delay_occurences_per_route_df = pd.DataFrame.from_dict(delay_occurences_per_route, orient ="index").reset_index().rename(columns = {"index": "Route", 0: "Delay_frequency"})
delay_occurences_per_route_df["Re"]  = delay_occurences_per_route_df.reset_index()["Route"].apply(lambda x: routes.get(x)["Direction of relation"].iloc[0])
delay_occurences_per_route_df

In [ ]:
delay_occurences_per_route_df["Delay_frequency"].plot.hist(bins = 100)

In [ ]:
delay_occurences_per_route_df["Delay_frequency"].plot.hist(cumulative = True, bins = 100)

In [ ]:
delay_occurences_per_route_df["Delay_frequency"].describe()

Intriguing that train routes that always have delays are not considered outliers. All routes but the the train routes from the output below are more likely to experience delays. We can also see this by calculating the p value and z-score

In [ ]:
def get_outlier(df, column):
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr = q3 - q1
    print(iqr)
    outlier = df[(df[column] > q3 + 1.5 * iqr) | (df[column] < q1 - 1.5 * iqr)]
    print(outlier)
    return outlier

df = get_outlier(delay_occurences_per_route_df, "Delay_frequency")


In [ ]:

import scipy.stats as stats
def calculate_z_score(df, column):
    df["z_score"] = (df[column] - df[column].mean()) / df[column].std(ddof=0)
    return df
def calculate_p_value(df):
    df["p_value"] = stats.norm.sf(abs(df["z_score"])) * 2
    return df

calculate_z_score(delay_occurences_per_route_df, "Delay_frequency")
pd.options.display.max_rows = 400000
calculate_p_value(delay_occurences_per_route_df)
delay_occurences_per_route_df

In [ ]:
delay_occurences_per_route_df.sort_values(by = ["Delay_frequency","Re"], ascending=False)


Maybe get the mean delay frequency per Re

In [ ]:
delay_occurences_per_route_df["test"] = delay_occurences_per_route_df.groupby("Re")["Delay_frequency"].transform("mean")
delays_per_relation = delay_occurences_per_route_df[["Re", "Delay_frequency"]]
delays_per_relation["avg_delay_freq"] = delays_per_relation.groupby("Re")["Delay_frequency"].transform("mean")
delays_per_relation = delays_per_relation.drop(columns = ["Delay_frequency"]).drop_duplicates()
delays_per_relation

In [ ]:
delays_per_relation

In [ ]:
get_outlier(delays_per_relation, "avg_delay_freq")


In [ ]:
delays_per_relation.plot.hist()

In [ ]:
routes.get(17209)

In [ ]:
delay_occurences_per_route_df[delay_occurences_per_route_df["p_value"] <0.05]